In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import matplotlib as mpl # graph
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # graph

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/Books.csv


In [2]:
#read all the input files
books = pd.read_csv("../input/book-recommendation-dataset/Books.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
users = pd.read_csv("../input/book-recommendation-dataset/Users.csv")
ratings = pd.read_csv("../input/book-recommendation-dataset/Ratings.csv")

In [4]:
books.head(8)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
#Selecting only useful information ( removing image URLs)
books = books.iloc[:,:5]

In [6]:
books.head(3)

,ISBN,Biiiook-Title,Book-Auiithor,Year-Oiiiiif-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [7]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
#Groupby to see which user has rated how many books
user_rating_count = ratings.groupby('User-ID').count().sort_values(by = 'ISBN', ascending = False)

In [10]:
user_rating_count.shape

(105283, 2)

In [11]:
#keep only those users who have given ratings to more than 100 books)
user_rating_count = user_rating_count[user_rating_count['ISBN']>100]

In [12]:
user_rating_count.shape

(1825, 2)

In [13]:
#Get counts of the ratings for each book
book_rating_count = ratings.groupby('ISBN').count().sort_values(by = 'Book-Rating', ascending = False)

In [14]:
book_rating_count.head(3)

,User-ID,Book-Rating
ISBN,,
0971880107,2502,2502
0316666343,1295,1295
0385504209,883,883
0060928336,732,732
0312195516,723,723
044023722X,647,647
0679781587,639,639
0142001740,615,615
067976402X,614,614


In [15]:
#only keep books with more than 50 ratings
book_rating_count = book_rating_count[book_rating_count['User-ID'] > 50]

In [16]:
ratings = ratings.merge(books, on = 'ISBN')

In [17]:
ratings.head(7)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


In [18]:
ratings.shape

(1031136, 7)

In [19]:
ratings = ratings[ratings['User-ID'].isin(user_rating_count.index)]

In [20]:
ratings.shape

(604854, 7)

In [21]:
ratings = ratings[ratings['ISBN'].isin(book_rating_count.index)]

In [22]:
ratings.shape

(116873, 7)

In [23]:
ratings.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
14,77480,034545104X,8,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


In [24]:
piv_table = ratings.pivot_table(columns = "User-ID", index = "ISBN", values = "Book-Rating")

In [25]:
piv_table.shape

(2101, 1793)

So we have 2101 books and 1792 users.


In [26]:
piv_table.iloc[:100, :100]

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,16161,16413,16488,16601,16634,16721,16795,16916,16943,16966
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002026478X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0020442203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
002542730X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
0028604199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0061097659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0061097853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0061097861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As you can see it is a very sparse table with most values as NaN. We will replace Nan to 0 to logically define that if  user A has given a rating of 0 we will not recommend that to similar users. This is true if user A has not read the book as well.

In [27]:
piv_table = piv_table.fillna(0)

In [28]:
pd.DataFrame(piv_table).isnull().sum().sum()

0

In [29]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(piv_table)

NearestNeighbors(algorithm='brute')

Let's list a few books at random and see which one wwe know. 

In [30]:
ratings.sample(frac = 0.001)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
195739,245963,0446364762,0,The Stars Shine Down,Sidney Sheldon,1993,Warner Books
93934,220278,0446672211,0,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books
462916,35859,038531292X,0,The Gift,Danielle Steel,1994,Delacorte Press
292878,218568,0385314698,6,The Sixteen Pleasures: A Novel,Robert Hellenga,1995,Delta
40786,190925,0553576801,0,The Perfect Husband,LISA GARDNER,1997,Bantam
...,...,...,...,...,...,...,...
245959,168064,0743211375,0,From a Buick 8 : A Novel,Stephen King,2002,Scribner
296126,264317,1573225517,0,High Fidelity,Nick Hornby,1996,Riverhead Books
185532,270713,0553278398,0,Prelude to Foundation (Foundation Novels (Pape...,ISAAC ASIMOV,1989,Spectra
24009,26593,0449221512,0,I Is for Innocent,Sue Grafton,1994,Fawcett Books


I had to sample multiple times to get a book that I have read. 'The Fellowship of the Ring' with ISBN 0618002227 at index 196979

In [31]:
piv_table[piv_table.index == '0618002227']

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,276018,276463,276680,276925,277427,277478,277639,278137,278188,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
0618002227,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
test_book = piv_table[piv_table.index == '0618002227'].values

In [33]:
test_book

array([[8., 0., 0., ..., 0., 0., 0.]])

In [34]:
model.kneighbors(test_book)

(array([[ 0.        , 41.76122604, 41.78516483, 41.78516483, 41.86884283]]),
 array([[1626, 1001, 2088, 2098, 1299]]))

From the doc we are getting distances and the 

In [35]:
_, closest = model.kneighbors(test_book)

In [36]:
for i in closest[:5]:
    print(i)

[1626 1001 2088 2098 1299]


In [37]:
for i in closest[:5][0]:
    print(books[books['ISBN'] == piv_table.index[i]])

             ISBN                                         Book-Title  \
12763  0618002227  반지의 제왕 (The Lord of the Ri...   

            Book-Author Year-Of-Publication                 Publisher  
12763  J. R. R. Tolkien                1999  Houghton Mifflin Company  
             ISBN                             Book-Title     Book-Author  \
18670  0440944597  The Chocolate War (Laurel Leaf Books)  ROBERT CORMIER   

      Year-Of-Publication    Publisher  
18670                1986  Laurel Leaf  
            ISBN Book-Title   Book-Author Year-Of-Publication  \
3028  1844262553       Free  Paul Vincent                2003   

               Publisher  
3028  Upfront Publishing  
            ISBN               Book-Title      Book-Author  \
8838  3423202327  MÃ?Â¶rder ohne Gesicht.  Henning Mankell   

     Year-Of-Publication Publisher  
8838                1999       Dtv  
            ISBN    Book-Title   Book-Author Year-Of-Publication    Publisher
2227  0451208811  Blood Orchid  St